In [7]:
%pip install pandas 
%pip install python-dotenv
%pip install -U scikit-learn
%pip install sun_position_calculator
%pip install requests
%pip install docxtpl
%pip install matplotlib
%pip install tabulate
%pip install pyxirr
%pip install openpyxl
%pip install folium
%pip install html2image

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [8]:
#local env variables
from dotenv import dotenv_values
config = dotenv_values(".env.local")

# Building references

## Address and general info🌎

## Energy Consumptions🔌

In [9]:
#cspell:disable


from models.components import Tech,Component
from models.consumption import Energetic
from models.inventory import Building, Project


project:Project = Project(
    title='Sistema PV MHNV OnGrid',
    building = Building(
        geolocation=(-33.047016, -71.621509),
        name='Edificio Carlos Porter MHNV',
        address='Calle Condell 1546',
        city='Valparaíso'),
    technology= [Tech.PHOTOVOLTAIC],
    consumption= {
        'description':'main',
        'energetic':Energetic.ELI,
        'cost_increment':8/100,
        'consumption':[
            (1051,"01-02-2023",97361),
            (1123,"02-03-2023",104031),
            (1157,"03-04-2023",107181),
            (1220,"04-05-2023",137911),
            (1299,"01-06-2023",146841),
            (1549,"04-07-2023",175101),
            (1353,"02-08-2023",152946),
            (1420,"01-09-2023",160519),
            (1283,"03-10-2023",145033),
            (1275,"03-11-2023",144128),
            (1346,"03-12-2023",152155),
            (1438,"03-01-2024",162554),

            ],
        }
    )

getting weather data...
api request URL https://power.larc.nasa.gov/api/temporal/hourly/point?Time=LST&parameters=T2M,ALLSKY_SFC_SW_DNI,ALLSKY_SFC_SW_DIFF,ALLSKY_SRF_ALB,SZA,WS10M&community=RE&latitude=-33.047016&longitude=-71.621509&start=20230101&end=20231231&format=JSON
getting currencies data...
set $1 Pesos chileno.. on USD$0.00 
set $1 UF............. on USD$39.03 
set $1 UTM............ on USD$68.44 
set $1 Euro........... on USD$1.05 
set $1 Pound.......... on USD$1.26 
set $1 Real........... on USD$0.17 
adding consumptions data...


# Integration 

## Machine Components 🎡

In [10]:
#cspell: disable
#repository of reusable components
from libs.repository import repoEquipment,panelRepo
from models.components import Specs
from models.econometrics import Currency,Cost
from models.geometry import Orientation
#clean previous load
project.components = {}

from IPython.display import clear_output

from models.energy_storage import Battery
from models.photovoltaic import PvInput
clear_output(wait=True)

# assembly projects components
project.add_generator(
    panelRepo['CS 655W'],# equipment
    PvInput(
        description='FV 655W MOD 01',
        quantity=9,
        orientation=Orientation(35,28),
        ),
    PvInput(
        description='FV 655W MOD 02',
        quantity=11,
        orientation=Orientation(20,28),
        ),
    PvInput(
        description='FV 655W MOD 03',
        quantity=4,
        orientation=Orientation(10,28),
    )
    
)

#nominal power
pj_power = project.nominal_power

# Energy process
project.add_component(
    'instalación',
    repoEquipment['Inverter']['DY 12kW H'],# inverter
    repoEquipment['Medidor']['FR 3F'],# lectura
    Component(
    description='eléctrica interior',
    specification=Specs(
        category='Obra',
        brand='conexionado',
        model='interiores'
        ),
    cost_per_unit=Cost(225_000,currency=Currency.CLP),
    quantity=int(project.nominal_power[0])
    ),
Component(
    description='est. de montaje',
    specification=Specs(
        category='Montaje',
        montaje='coplanar',
        ),
    cost_per_unit=Cost(45_000,Currency.CLP),
    quantity=project.n_generator
    ),
)

# BESS & Storage
project.add_storage(
    'almacenamiento',4,'8/5',
    repoEquipment['Monitor']['VC 700'],
    repoEquipment['Regulator']['VT 45A']
    )

# obras y accesorios
project.add_component(
    'Obras',
    Component(
        description='Faenas',
        specification=Specs(
            category='Obra',
            brand='equipamiento',
            model='provisorio'),
        cost_per_unit=Cost(18_750,Currency.CLP),
        quantity=project.n_generator
        ),
    Component(
        description='Capacitación',
        specification=Specs(
            category='Obra',
            brand='Uso y',
            model='mantenimiento',
            taller='2 Hrs',
            manual='3 u. impreso',
            ),
        cost_per_unit=Cost(250_000,Currency.CLP),
        ),
    Component(
        description='Letrero',
        specification=Specs(
            category='Obra',
            brand='pliego',
            model='estructura',
            specs_url='#pliegos-técnicos',
            dim='3.6m x 1.5m',
            ),
        cost_per_unit=Cost(450_000,Currency.CLP),
        )
)

inside cost pv :  208547.06 Currency.CLP
inside cost pv :  208547.06 Currency.CLP
inside cost pv :  208547.06 Currency.CLP


In [11]:
#calculate bucket overloads
project.bucket.set_overloads(gastos_gral=15,utilidad=10)


In [13]:
project._performance

,month,consumption,unit_cost,generation,netbilling,savings,benefits,CO2 kg
0,1,1438,148.162799,2690.763800,1252.763800,1438.000000,213058.104379,548.286689
1,2,1051,123.002997,2283.255837,1232.255837,1051.000000,129276.150286,465.250344
2,3,1123,127.683891,2162.643656,1039.643656,1123.000000,143389.009211,440.673659
3,4,1157,129.894312,1501.194060,344.194060,1157.000000,150287.719526,305.892595
4,5,1220,133.990094,1124.717462,0.000000,1124.717462,150700.998569,229.179393
5,6,1299,139.126074,954.973895,0.000000,954.973895,132861.768930,194.591393
6,7,1549,155.379176,979.817273,0.000000,979.817273,152243.200170,199.653633
7,8,1353,142.636744,1223.134070,0.000000,1223.134070,174463.861360,249.233370
8,9,1420,146.992575,1522.180656,102.180656,1420.000000,208729.456904,310.168954
9,10,1283,138.085876,2062.576214,779.576214,1283.000000,177164.178450,420.283298


# Write Document 📒


In [8]:
import os
PATH = 'build/'+f'r_{project.building.city.lower()[:3]}_{project.building.name}/'

if not os.path.exists(PATH):
    #create
    os.makedirs(PATH)

### Saving results on EXCEL ❎

In [9]:
from models.plotter import to_table
to_table(project,PATH)

### Generate Context->Doc📄✍️

In [10]:
from docxtpl import DocxTemplate # type: ignore
#init doc
#assembly proxy object 
memory_report = DocxTemplate("templates/memory_template.docx")
bidding_report = DocxTemplate("templates/bidding_template.docx")

## Write params and tables

In [11]:
#loading context
#cspell: disable
memory_report.render(project.context(template=memory_report))
bidding_report.render(project.context(template=bidding_report))


## Insert Plots 📊

In [ ]:
#dep
from models.plotter import plotter
plotter(project,PATH)

In [ ]:
#set into doc
plot_list = [
    'plot_consumption_forecast',
    'plot_irradiance',
    'plot_temperature',
    'plot_components',
    'plot_components_irr',
    'plot_components_production',
    'plot_production_performance',
    'plot_performance_frequency',
    'plot_flux',
    'map_location'
]

for plot in plot_list:
    memory_report.replace_pic(plot,PATH+f'{plot}.png')
    print('replaced plot:',plot)

In [ ]:
from datetime import datetime
memory_report.save(PATH+f"reporte_memoria_calculo.docx")
bidding_report.save(PATH+f"reporte_pliegos_técnicos.docx")
print('work',project,'finish at: ',datetime.now())